The idea is to create features that include many stocks, and targets that are the future prices for a range of days, as well as the OHLC for each of those days.

The plan is:
* prototype feature/target creation and neural net with OHLC for a single day
* extend prototype to predict future values for a range of days
* extend prototype to include multiple stocks
* convert to a function
* put in a file and test functions

In [1]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd

In [4]:
sorted_old = pd.read_csv('old_good_sorted_by_ewm.csv')

In [5]:
top_stocks = sorted_old.loc[:100, 'ticker'].tolist() + ['SPY', 'UPRO', 'QQQ', 'TQQQ', 'DIA', 'UBT']

# prototype feat/targ creation and neural net with OHLC for single day

In [ ]:
future_days = 1
history_days = 5
for 